In [1]:
from strands import Agent
from strands.models.anthropic import AnthropicModel
from strands.models.ollama import OllamaModel
from dotenv import load_dotenv
from pydantic import BaseModel, Field
import os

load_dotenv()

anthropic_api_key=os.getenv("ANTHROPIC_API_KEY")


In [2]:
ollama_model = OllamaModel(model_id="qwen3:8b", host="http://localhost:11434")
anthropic_model = AnthropicModel(
    client_args={
        "api_key": anthropic_api_key,
    },
    # **model_config
    max_tokens=1028,
    model_id="claude-sonnet-4-20250514",
    params={
        "temperature": 0.1,
    }
)

In [6]:
sample_agent = Agent(model=ollama_model, system_prompt="Extract the name of the person and the address")

class Info(BaseModel):
    name: str = Field(description="The name of the person")
    door_no: str = Field(description="The door number of the person")
    street: str = Field(description="The street name of the person")
    city: str = Field(description="The city of the person")
    province: str = Field(description="The province of the person")
    country: str = Field(description="The country of the person")
    

In [7]:
result = sample_agent.structured_output(Info,"srivatsa lives in 50 graydon hall drive, north york, ontario, canada")

In [8]:
type(result)

__main__.Info

In [10]:
result.model_dump()

{'name': 'Srivatsa',
 'door_no': '50',
 'street': 'graydon hall drive',
 'city': 'north york',
 'province': 'ontario',
 'country': 'canada'}

In [11]:
result = sample_agent("srivatsa lives in 50 graydon hall drive, north york, ontario, canada")
type(result)


<think>
Okay, let's see. The user wants me to extract the name of the person and their address from the given text. The input is "srivatsa lives in 50 graydon hall drive, north york, ontario, canada" repeated twice. 

First, I need to identify the person's name. The text starts with "srivatsa," which looks like a first name. Since there's no other name mentioned, I can assume that's the person's name.

Next, the address part. The address is "50 graydon hall drive, north york, ontario, canada." I need to parse this into standard address components. Typically, an address includes street number, street name, city, state, and country. Here, "50 graydon hall drive" is the street address. Then "north york" is the city. "Ontario" is the province, and "canada" is the country. 

Wait, but sometimes "North York" is a city in Ontario, Canada. So the address components should be structured as: Street: 50 Graydon Hall Drive, City: North York, Province: Ontario, Country: Canada. 

I should make sure

strands.agent.agent_result.AgentResult

In [14]:
result

AgentResult(stop_reason='end_turn', message={'role': 'assistant', 'content': [{'text': '<think>\nOkay, let\'s see. The user wants me to extract the name of the person and their address from the given text. The input is "srivatsa lives in 50 graydon hall drive, north york, ontario, canada" repeated twice. \n\nFirst, I need to identify the person\'s name. The text starts with "srivatsa," which looks like a first name. Since there\'s no other name mentioned, I can assume that\'s the person\'s name.\n\nNext, the address part. The address is "50 graydon hall drive, north york, ontario, canada." I need to parse this into standard address components. Typically, an address includes street number, street name, city, state, and country. Here, "50 graydon hall drive" is the street address. Then "north york" is the city. "Ontario" is the province, and "canada" is the country. \n\nWait, but sometimes "North York" is a city in Ontario, Canada. So the address components should be structured as: Stree

#### human in loop example

In [ ]:
from strands_tools import handoff_to_user
agent = Agent(model=ollama_model, tools=[handoff_to_user])
result = agent("write a short story about a cat, and handoff the task to the user for review by using the handoff_to_user tool. Note: you must call this tool to execute the handoff. Do not print anything else.")
response = agent.tool.handoff_to_user(
    message="I need your approval to proceed",
    breakout_of_loop=False
)

<think>
Okay, the user wants me to write a short story about a cat and then hand off the task to them for review using the handoff_to_user tool. Let me start by crafting a simple story.

Hmm, the story should be engaging but concise. Let's think of a little cat with a unique trait. Maybe a cat named Whiskers who can talk to birds. That adds a magical element. 

I need to set the scene. Perhaps a sunny afternoon in a garden. Whiskers meets a bird, they have a conversation, and then something happens. Maybe the bird invites him to a feast. Then, a problem arises, like a storm approaching. They work together to solve it. 

Wait, the user wants the story to be short, so I should keep it to a few paragraphs. Let me outline: introduction of Whiskers, his ability, meeting the bird, the feast, the storm, and a happy ending. 

Now, after writing the story, I need to use the handoff_to_user function. The parameters require a message and whether to breakout of the loop. Since the user wants revie

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│                                                                                                                 │
│  🤝 AGENT REQUESTING USER HANDOFF                                                                               │
│                                                                                                                 │
│  Here's the story:                                                                                              │
│                                                                                                                 │
│  Whiskers, a curious tabby cat, spent his afternoons exploring the sun-dappled garden of Mrs. Thompson. One     │
│  day, he discovered a hidden path leading to a secret orchard where birds sang in harmony. A sparrow named Pip  │
│  welcomed him, revealing the birds had been gathering fruit for a feast. As Whiskers joined the celebration, a  │
│  sudden storm approached. Together, they guided the birds to safety, their friendship bridging species. The     │
│  next morning, the orchard bloomed brighter, a testament to their unlikely bond.                                │
│                                                                                                                 │
│  Please review this story and let me know if you'd like any edits!                                              │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Your response:






Your response:






                 

In [12]:
result

AgentResult(stop_reason='tool_use', message={'role': 'assistant', 'content': [{'toolUse': {'toolUseId': 'handoff_to_user', 'name': 'handoff_to_user', 'input': {'breakout_of_loop': True, 'message': "Here's your short story about a cat. Please review it: \n\nWhiskers was a curious tabby who loved exploring the garden. One day, she chased a butterfly through the flowers, climbed the tallest sunflower, and returned home with a tiny moth as her new friend. The end! Let me know if you need changes."}}}, {'text': '<think>\nOkay, the user wants me to write a short story about a cat and then hand off the task to them for review using the handoff_to_user tool. Let me start by crafting a simple story.\n\nFirst, I need to make sure the story is concise. Let\'s think of a little plot. Maybe a cat named Whiskers who loves adventures. Something like Whiskers exploring the garden, meeting a butterfly, and then coming home. Keep it friendly and short.\n\nNow, after writing the story, I need to use the 